In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext, SparkConf

In [3]:
conf = SparkConf().setMaster("yarn").setAppName("SGD")

In [4]:
sc = SparkContext.getOrCreate (conf = conf)
sc.getConf().getAll()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/02 07:16:54 WARN DataStreamer: Exception in createBlockOutputStream blk_1073742188_1367
java.io.IOException: Got error, status=ERROR, status message , ack with firstBadLink as 192.1.1.20:9866
	at org.apache.hadoop.hdfs.protocol.datatransfer.DataTransferProtoUtil.checkBlockOpStatus(DataTransferProtoUtil.java:128)
	at org.apache.hadoop.hdfs.protocol.datatransfer.DataTransferProtoUtil.checkBlockOpStatus(DataTransferProtoUtil.java:104)
	at org.apache.hadoop.hdfs.DataStreamer.createBlockOutputStream(DataStreamer.java:1827)
	at org.apache.hadoop.hdfs.DataStreamer.nextBlockOutputStream(DataStreamer.java:1728)
	at org.apache.hadoop.hdfs.DataStreamer.run(DataStreamer.java:713)
23/01/02 07:16:54 WARN DataStreamer: Abandoning BP-1190023728-192.1.1.10-1671092909957:blk_1073742188_1367
23/01/02 07:16:54 WARN DataStreamer: Excluding datanode DatanodeInfoWithStorage[192.1.1.20:9866,DS-2ea39210-7e6c-47f0-b0f8-4cabbce6fcd4,DISK]
23/01/02 07:16:55 WARN DataStreamer: Slow waitForAckedSeqno took 669

[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.app.startTime', '1672643727503'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES',
  'http://master:8088/proxy/application_1672643310779_0002'),
 ('spark.app.submitTime', '167264372

In [5]:
import seaborn as sns
import re
import csv
import math
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib  import cm
import numpy as np

In [6]:
createType = "csv_read_chunk"
database = "Windsor_Housing_large"
num_of_partition = 2

In [7]:
def read_csvv(dataset_name):
    df = pd.read_csv("{}.csv".format(dataset_name))
    return df

In [8]:
df_origin = read_csvv(database)

In [9]:
df_origin

,Unnamed: 0,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,0,42000.0,5850,3,1,2,yes,no,yes,no,no,1,no
1,1,38500.0,4000,2,1,1,yes,no,no,no,no,0,no
2,2,49500.0,3060,3,1,1,yes,no,no,no,no,0,no
3,3,60500.0,6650,3,1,2,yes,yes,no,no,no,0,no
4,4,61000.0,6360,2,1,1,yes,no,no,no,no,0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,2541,254000.0,15516,4,3,2,yes,no,no,no,yes,0,no
2542,2542,210200.0,21116,4,3,1,yes,no,no,no,no,0,no
2543,2543,478100.0,22814,5,1,2,yes,no,no,no,no,2,no
2544,2544,1988100.0,62908,8,6,2,yes,yes,yes,yes,no,2,no


In [10]:
df = df_origin.copy()

In [11]:
from sklearn.model_selection import train_test_split

df, test_df = train_test_split(df, test_size=0.3)

In [12]:
from sklearn.preprocessing import StandardScaler

featureScaler = StandardScaler()
df[['lotsize']] = featureScaler.fit_transform(df[['lotsize']])

priceScaler = StandardScaler()
df[['price']] = priceScaler.fit_transform(df[['price']])


In [13]:
df

,Unnamed: 0,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
855,855,-0.558410,-0.868905,5,2,3,yes,no,no,no,yes,0,yes
1788,1788,-0.077950,0.037293,5,1,2,yes,no,no,no,no,1,no
153,153,-1.001249,-1.222585,3,1,2,no,no,no,no,no,1,no
1010,1010,-0.295970,-0.206795,2,2,2,yes,no,no,no,yes,1,no
2225,2225,0.310045,-0.251496,5,1,4,yes,yes,no,no,yes,2,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...
860,860,-0.338124,-0.255031,2,2,2,yes,yes,no,yes,no,1,no
775,775,-0.230700,-0.860577,2,3,2,yes,yes,no,no,yes,0,no
510,510,-0.874335,-1.140411,3,1,2,yes,yes,yes,no,no,2,no
1116,1116,-0.208037,0.214369,2,1,1,no,no,no,no,no,0,no


In [14]:
category_cols = ['driveway', 'recroom', 'fullbase', 'gashw', 'airco', 'prefarea']

In [15]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [16]:
for cate in category_cols:
    df[cate] = label_encoder.fit_transform(df[cate])

In [17]:
df

,Unnamed: 0,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
855,855,-0.558410,-0.868905,5,2,3,1,0,0,0,1,0,1
1788,1788,-0.077950,0.037293,5,1,2,1,0,0,0,0,1,0
153,153,-1.001249,-1.222585,3,1,2,0,0,0,0,0,1,0
1010,1010,-0.295970,-0.206795,2,2,2,1,0,0,0,1,1,0
2225,2225,0.310045,-0.251496,5,1,4,1,1,0,0,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
860,860,-0.338124,-0.255031,2,2,2,1,1,0,1,0,1,0
775,775,-0.230700,-0.860577,2,3,2,1,1,0,0,1,0,0
510,510,-0.874335,-1.140411,3,1,2,1,1,1,0,0,2,0
1116,1116,-0.208037,0.214369,2,1,1,0,0,0,0,0,0,0


In [18]:
columns = list(df.columns.values)

In [19]:
columns

['Unnamed: 0',
 'price',
 'lotsize',
 'bedrooms',
 'bathrms',
 'stories',
 'driveway',
 'recroom',
 'fullbase',
 'gashw',
 'airco',
 'garagepl',
 'prefarea']

In [20]:
wholeRDD = sc.parallelize(df.values.tolist(), num_of_partition)

In [21]:
wholeRDD.take(5)

23/01/02 07:18:42 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


[[855.0,
  -0.5584100716254294,
  -0.8689047364116417,
  5.0,
  2.0,
  3.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0],
 [1788.0,
  -0.07794978073488291,
  0.037293014306491956,
  5.0,
  1.0,
  2.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0],
 [153.0,
  -1.0012494152103955,
  -1.2225850375588492,
  3.0,
  1.0,
  2.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0],
 [1010.0,
  -0.2959699693371026,
  -0.20679510112317784,
  2.0,
  2.0,
  2.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0],
 [2225.0,
  0.31004456738050185,
  -0.251496143182876,
  5.0,
  1.0,
  4.0,
  1.0,
  1.0,
  0.0,
  0.0,
  1.0,
  2.0,
  0.0]]

In [22]:
columns[0] = 'id'
print(columns)

['id', 'price', 'lotsize', 'bedrooms', 'bathrms', 'stories', 'driveway', 'recroom', 'fullbase', 'gashw', 'airco', 'garagepl', 'prefarea']


In [23]:
c = {name: idx  for idx, name in enumerate(columns)}
print(c)

{'id': 0, 'price': 1, 'lotsize': 2, 'bedrooms': 3, 'bathrms': 4, 'stories': 5, 'driveway': 6, 'recroom': 7, 'fullbase': 8, 'gashw': 9, 'airco': 10, 'garagepl': 11, 'prefarea': 12}


In [24]:
c

{'id': 0,
 'price': 1,
 'lotsize': 2,
 'bedrooms': 3,
 'bathrms': 4,
 'stories': 5,
 'driveway': 6,
 'recroom': 7,
 'fullbase': 8,
 'gashw': 9,
 'airco': 10,
 'garagepl': 11,
 'prefarea': 12}

In [25]:
print("Number of training item = ", wholeRDD.count())
print("Number of columns = ", len(columns))

Number of training item =  1782
Number of columns =  13


In [26]:
feature_num = len(columns) - 2 # except id and price columns
feature_num = feature_num + 1 # bias

In [27]:
def parseData(data):
    try:
        t = float(data)        
        return t
    except:
        return data
    

def cleanData(line):
    new_line = list(line)
    for idx, value in enumerate(new_line):
        if value == '':
            new_line[idx] = None
        
        if new_line[idx] != None:
            new_line[idx] = parseData(new_line[idx])
    return tuple(new_line)
            

wholeRDD = wholeRDD.map(lambda line: cleanData(line))

In [28]:
wholeRDD.take(5)

[(855.0,
  -0.5584100716254294,
  -0.8689047364116417,
  5.0,
  2.0,
  3.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0),
 (1788.0,
  -0.07794978073488291,
  0.037293014306491956,
  5.0,
  1.0,
  2.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0),
 (153.0,
  -1.0012494152103955,
  -1.2225850375588492,
  3.0,
  1.0,
  2.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0),
 (1010.0,
  -0.2959699693371026,
  -0.20679510112317784,
  2.0,
  2.0,
  2.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0),
 (2225.0,
  0.31004456738050185,
  -0.251496143182876,
  5.0,
  1.0,
  4.0,
  1.0,
  1.0,
  0.0,
  0.0,
  1.0,
  2.0,
  0.0)]

In [29]:
def checkNone(line):
    for i in line:
        if i == None:
            return False
    
    return True

In [30]:
wholeRDD = wholeRDD.filter(lambda line: checkNone(line))

In [31]:
def addBias(line):
    new_line = list(line)
    
    # add bias 1
    new_line.append(1)
    
    # Remove id
    new_line = new_line[1:]
    
    # Add 1 col for different of label and predicted values
    new_line.append(1)
    
    return tuple(new_line)

In [32]:
# Add bias as new column
wholeRDD = wholeRDD.map(lambda line:addBias(line))

In [33]:
wholeRDD.take(5)

[(-0.5584100716254294,
  -0.8689047364116417,
  5.0,
  2.0,
  3.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1,
  1),
 (-0.07794978073488291,
  0.037293014306491956,
  5.0,
  1.0,
  2.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1,
  1),
 (-1.0012494152103955,
  -1.2225850375588492,
  3.0,
  1.0,
  2.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1,
  1),
 (-0.2959699693371026,
  -0.20679510112317784,
  2.0,
  2.0,
  2.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  1,
  1),
 (0.31004456738050185,
  -0.251496143182876,
  5.0,
  1.0,
  4.0,
  1.0,
  1.0,
  0.0,
  0.0,
  1.0,
  2.0,
  0.0,
  1,
  1)]

In [34]:
def dot_product_2_vec(vec1, vec2):
    summ = 0
    
    assert len(vec1) == len(vec2)
    for i in range(len(vec1)):
        summ = summ + vec1[i] * vec2[i]
    
    return summ

def add_new_diff(line, value):
    new_line = list(line)
    new_line[len(new_line)-1] = value
    return tuple(new_line)
    
def dot_product_diff_mat_vec(rdd , vec):
    return rdd.map(lambda line: add_new_diff(line, dot_product_2_vec(line[1:-1] , vec) - line[0]))

In [35]:
def find_grad(rdd, idx_weight):
    return rdd.map(lambda line: line[idx_weight] * line[-1]).reduce(lambda a,b : a + b)

In [36]:
iterations = 150
costs = [0 for i in range(iterations)]
learning_rate = 0.1

init_weight = np.random.normal(0, 1, feature_num).tolist()
assert len(init_weight) == feature_num

training_num = wholeRDD.count()

print(training_num)
best_loss = math.inf

train_rdd = wholeRDD
for i in range(iterations):

    train_rdd = dot_product_diff_mat_vec(train_rdd, init_weight)
    
    # compute loss
    cost_value = train_rdd.map(lambda x : x[-1] * x[-1]).reduce(lambda a, b: a + b) * (1 / training_num) * (1/2)
   
    print("[Loss {}/{}] {}".format(i, iterations, cost_value))

    costs[i] = cost_value
        
    # update weights
    for idx in range(len(init_weight)):
        init_weight[idx] = init_weight[idx] - learning_rate * (1 / training_num )* find_grad(train_rdd, idx + 1)
        
    if costs[i] < best_loss:
        best_loss = costs[i]

1782


[Loss 0/150] 11.169561456408477


23/01/02 07:19:13 WARN AccumulatorContext: Attempted to access garbage collected accumulator 277
23/01/02 07:19:13 WARN AccumulatorContext: Attempted to access garbage collected accumulator 277
23/01/02 07:19:13 WARN AccumulatorContext: Attempted to access garbage collected accumulator 278
23/01/02 07:19:13 WARN AccumulatorContext: Attempted to access garbage collected accumulator 278
23/01/02 07:19:13 WARN AccumulatorContext: Attempted to access garbage collected accumulator 284
23/01/02 07:19:13 WARN AccumulatorContext: Attempted to access garbage collected accumulator 284


[Loss 1/150] 2.272878407425721
[Loss 2/150] 1.4286847386591415


[Loss 3/150] 1.2226275717348065
[Loss 4/150] 1.0768500251825233
[Loss 5/150] 0.9810712845396615
[Loss 6/150] 0.9119863759549895
[Loss 7/150] 0.8584763744603429
[Loss 8/150] 0.8143825153411067
[Loss 9/150] 0.776348341040149
[Loss 10/150] 0.7425120949973588
[Loss 11/150] 0.7118144122917446
[Loss 12/150] 0.6836265427427745
[Loss 13/150] 0.6575522150771069
[Loss 14/150] 0.6333224527230911
[Loss 15/150] 0.6107399049997871
[Loss 16/150] 0.589649381710937


...

[Loss 143/150] 0.22283103834091805


[Loss 144/150] 0.22266621345465382


[Loss 145/150] 0.22250408371589459
[Loss 146/150] 0.22234457866628157
[Loss 147/150] 0.22218763043097947


[Loss 148/150] 0.22203317360452046


[Loss 149/150] 0.22188114514230983


In [37]:
def predict(df_t, weight):
    
    df_test = df_t.copy()
    
    # preprocess
    df_test[['lotsize']] = featureScaler.transform(df_test[['lotsize']])
    
    for cate in category_cols:
        df_test[cate] = label_encoder.transform(df_test[cate])
        
    size = len(df_test)
    
    # predict
    df_test["bias"] = [1 for i in range(size)]
    
    print(df_test.columns)
    assert len(df_test.columns) == len(weight) + 2
    
    # predict
    df_test['predict_price'] = df_test.apply(lambda line: dot_product_2_vec(line[2:] , weight), axis=1)
    
    # rescale price to origin
    df_test['predict_price_normalized'] = priceScaler.inverse_transform([df_test['predict_price']])[0]
    return df_test

In [38]:
df_test_result = predict(test_df, init_weight)

Index(['Unnamed: 0', 'price', 'lotsize', 'bedrooms', 'bathrms', 'stories',
       'driveway', 'recroom', 'fullbase', 'gashw', 'airco', 'garagepl',
       'prefarea', 'bias'],
      dtype='object')


In [39]:
df_test_result[['price_scaled']] = priceScaler.transform(df_test_result[['price']])

In [40]:
df_test_result

,Unnamed: 0,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea,bias,predict_price,predict_price_normalized,price_scaled
833,833,328200.0,-0.267758,2,2,2,1,0,0,0,0,2,0,1,-0.365739,182207.518952,0.295993
1396,1396,400400.0,0.372119,5,4,3,1,0,0,1,0,1,0,1,1.311432,552227.910374,0.623250
1089,1089,339800.0,-0.158087,5,1,1,0,0,1,0,0,0,0,1,0.164379,299162.971476,0.348572
1475,1475,232900.0,0.053712,4,2,2,1,0,0,0,1,0,0,1,0.047706,273422.408119,-0.135968
1923,1923,263500.0,-0.265323,1,1,2,1,0,0,0,0,2,0,1,-0.613477,127551.138689,0.002731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,490,50000.0,-0.488514,3,1,1,1,0,0,0,0,0,0,1,-0.626932,124582.477887,-0.964988
1993,1993,218500.0,0.028966,2,1,2,1,1,1,0,0,0,1,1,-0.218875,214608.834554,-0.201238
1443,1443,219300.0,0.487917,1,7,2,0,0,0,0,0,0,0,1,1.525679,599495.318191,-0.197612
1908,1908,208000.0,-0.493070,2,1,3,1,0,1,0,1,2,0,1,-0.471500,158874.296075,-0.248830


In [41]:
def calRMSE(vec1, vec2, n):
    summ = 0
    for i in range(n):
        summ += (vec1[i] - vec2[i])**2
    return summ * 1/(2*n)      

In [42]:
n = len(df_test_result)
print("Evaluation on RMSE (normalized form) = ", calRMSE(df_test_result['price'].tolist(), df_test_result['predict_price_normalized'].tolist(), n) )
print("Evaluation on RMSE (scaled form) = ", calRMSE(df_test_result['predict_price'].tolist(), df_test_result['price_scaled'].tolist(), n) )

Evaluation on RMSE (normalized form) =  11397614271.35817
Evaluation on RMSE (scaled form) =  0.23416243433690245


RMSE of orignal form (normalized form) is obviously high because price is large too, which make our gradients and errors big too. To prevent this, I scaled data with StandardScaler with mean 0 and std 1. 

The result is column 'predict_price_normalized'